In [23]:
import pandas as pd
from pathlib import Path

In [24]:
df = pd.read_csv("../data/reuters_headlines.csv")
df['Headlines']

0        TikTok considers London and other locations fo...
1        Disney cuts ad spending on Facebook amid growi...
2        Trail of missing Wirecard executive leads to B...
3        Twitter says attackers downloaded data from up...
4        U.S. Republicans seek liability protections as...
                               ...                        
32765    Malaysia says never hired British data firm at...
32766    Prosecutors search Volkswagen headquarters in ...
32767     McDonald's sets greenhouse gas reduction targets
32768    Pratt & Whitney to deliver spare A320neo engin...
32769    UK will always consider ways to improve data l...
Name: Headlines, Length: 32770, dtype: object

In [27]:
text = ' <> '.join(df['Headlines'].tolist())
text[0:201]

'TikTok considers London and other locations for headquarters <> Disney cuts ad spending on Facebook amid growing boycott: WSJ <> Trail of missing Wirecard executive leads to Belarus, Der Spiegel report'